# Rental Price Analysis of India's IT Capital - Pune, MH, IND

**Problem Defination** 

Pune is IT capital of India  & a city with 10,089,916 population. Pune is one of the larger employment producing city. So Most of peoples are living in rental houses & appartment.  so I'm going to predict the housing rental prices of areas in city.

you can check my dataset is here :

1. Rental Price Data - [kaggle](https://www.kaggle.com/anantsakhare/rental-price-of-indias-it-capital-pune-mh-ind) 
2. Housing data Scrapper - [github](https://github.com/senhorinfinito/scrappers/blob/main/rental_analysis/train.csv)

## Exploring dataset
### Import Libraries

In [ ]:
## Import Essential Libraries 
!pip install geocoder

import os #for file part
from math import * # for mathmatically relations & computation  
from scipy.stats import * # Stat testing
import numpy as np # array operations 
import re # working on text data 
import pandas as pd # dataframe operations
import matplotlib.pyplot as plt # visualisations
import seaborn as sns # Viz
import geocoder # Geographical Data analysis
import geopy # Geographical data
from geopy.geocoders import Nominatim  # Geographical data

# All machine leanring libraries
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
### Load data into a dataframe
train = pd.read_csv('/kaggle/input/rental-price-of-indias-it-capital-pune-mh-ind/train.csv', encoding = 'utf-8')

**Know more about dataset** 

In [ ]:
train.shape

Based on data shape of data data has around 10884 sample with 30 features. 

In [ ]:
train.info() # lookin for dtypes , null values inside data

In [ ]:
train.isnull().sum()

In [ ]:
train.describe() # A brief stat of data

In [ ]:
df = train.copy() # createing a copy of original data of actual cleaning & feature engineering puppose

**Now we have a proper dataset copy so we can work on that** 

In [ ]:
df.head(3)

## Few Feature Engineering on Target variables

In [ ]:
## Warning / NOTe
## few working requied on rent here intial
print('shape before', df.shape)
df.drop(df[df['rent'] >= 123456789].index, axis = 0, inplace = True) # this values is added by purpose while doing data cleaning 
print('shape after', df.shape)

In [ ]:
df.area.replace(0, np.nan, inplace =  True)
df.area.isnull().sum()

pd.pivot_table(df, values= 'area', columns='bedroom', aggfunc= 'mean').T
pd.pivot_table(df, values= 'area', columns='bedroom', aggfunc= 'median').T

m1 = df['bedroom']  == 1
m2 = df['bedroom']  == 2
m3 = df['bedroom']  == 3
m4 = df['bedroom']  == 4
m5 = df['bedroom']  == 5
m6 = df['bedroom']  == 6
df.loc[m1, 'area'] = df.loc[m1,'area'].fillna(df.loc[m1,'area'].median())
df.loc[m2, 'area'] = df.loc[m2,'area'].fillna(df.loc[m2,'area'].median())
df.loc[m3, 'area'] = df.loc[m3,'area'].fillna(df.loc[m3,'area'].median())
df.loc[m4, 'area'] = df.loc[m4,'area'].fillna(df.loc[m4,'area'].median())
df.loc[m5, 'area'] = df.loc[m5,'area'].fillna(df.loc[m5,'area'].median())
df.loc[m6, 'area'] = df.loc[m6,'area'].fillna(df.loc[m6,'area'].median())



## Exploratory Data Analysis 

Convert all variable into following categories
1. Binary variables data
2. Ordinal Varibles 
3. continous  data
4. target Variable

In [ ]:
num_columns = df.describe().columns
categorical_cols = df.describe(include= 'object').columns

In [ ]:
# checking for binary data & oridinal & continous_data 
for i in df.columns:
    print(len(df[i].unique()), end = '    ')
# range varies form 2 to  6387


In [ ]:
binary_variables = [i  for i in df.columns  if (len(df[i].unique()) == 2)]

In [ ]:
ordinal_variables= [i for i in df.columns if ((len(df[i].unique()) > 2 ) and (len(df[i].unique()) <= 25))]

In [ ]:
continous_variable = [i for i in df.columns if ((len(df[i].unique()) > 25))]
## Remove rent from data
continous_variable = continous_variable[:-1]

In [ ]:
target_variable = 'rent'

### Start with Binary Variable Visualisation.


In [ ]:
print(binary_variables, end = ' ')

('element in binary category', len(binary_variables))

In [ ]:
df.gate_community.value_counts().index

In [ ]:
fig, ax = plt.subplots(4,3, figsize = (20,30), sharey = True)
fig.tight_layout()
fig.subplots_adjust(top=0.95)

sns.set_style('white')
fig.suptitle('Binary Variables- Counts', size = 30)
sns.barplot(ax= ax[0,0], data =df , x = df.gate_community.value_counts().index, y = df.gate_community.value_counts().values,palette= 'muted' )
ax[0,0].set_title('Gate Facility', fontsize = 15)
sns.barplot(ax= ax[0,1], data =df , x = df.corner_pro.value_counts().index, y = df.corner_pro.value_counts().values,palette= 'pastel' )
ax[0,1].set_title('Conrner Property', fontsize = 15)
sns.barplot(ax= ax[0,2], data =df , x = df.wheelchairadption.value_counts().index, y = df.wheelchairadption.value_counts().values,palette= 'muted' )
ax[0,2].set_title('Wheel Chair Facility',   fontsize = 15)
sns.barplot(ax= ax[1,0], data =df , x = df.petfacility.value_counts().index, y = df.petfacility.value_counts().values,palette= 'husl' )
ax[1,0].set_title('Pet Friendly',  fontsize = 15)
sns.barplot(ax= ax[1,1], data =df , x = df.lightbill.value_counts().index, y = df.lightbill.value_counts().values,palette= 'Set2' )
ax[1,1].set_title('Electricity Bill Included',  fontsize = 15)
sns.barplot(ax= ax[1,2], data =df , x = df.no_room.value_counts().index, y = df.no_room.value_counts().values,palette= 'flare' )
ax[1,2].set_title('No other rooms',  fontsize = 15)
sns.barplot(ax= ax[2,0], data =df , x = df.pooja_room.value_counts().index, y = df.pooja_room.value_counts().values,palette= 'ch:s=.25,rot=-.25' )
ax[2,0].set_title('Pooja/ Holy Room',  fontsize = 15)
sns.barplot(ax= ax[2,1], data =df , x = df.study_room.value_counts().index, y = df.study_room.value_counts().values,palette= 'muted' )
ax[2,1].set_title('Study_Room',  fontsize = 15)
sns.barplot(ax= ax[2,2], data =df , x = df.others.value_counts().index, y = df.others.value_counts().values )
ax[2,2].set_title('other Rooms',  fontsize = 15)
sns.barplot(ax= ax[3,0], data =df , x = df.servant_room.value_counts().index, y = df.servant_room.value_counts().values,palette= 'flare' )
ax[3,0].set_title('Servent Room',  fontsize = 15)
sns.barplot(ax= ax[3,1], data =df , x = df.store_room.value_counts().index, y = df.store_room.value_counts().values,palette= 'husl' )
ax[3,1].set_title('Store Room',  fontsize = 15)
# sns.barplot(ax=ax[3,2], x = df.index, y = df.rent)
fig.delaxes(ax[3][2])
fig.savefig('my.jpg')

In [ ]:
target =  'rent'
for feature in binary_variables:
    df.groupby(feature)[target].median().plot.bar()
    
    plt.xlabel(feature)
    plt.ylabel(target)
    plt.title('{} vs log({})_median'.format(feature, target))
    plt.show()

**Based on above plots I found that if Facility is available then rent for home is always higher**

### Ordinal Variables 

In [ ]:
print(ordinal_variables, end = ' ')
('Number of ordinal features is', len(ordinal_variables))

In [ ]:
### working on oultiers which I found while working on previously on porject
df.groupby('bedroom').count()
# we have more than 6  bedrooms in single homes  so we will drop to reduce outliers  totals around 20
print('before shape',df.shape)
df.drop(df[df['bedroom']>= 6].index, axis = 0, inplace = True)
print('after droping rows the shape is ', df.shape)

In [ ]:
print('before replacement', df.furnishing.unique())
df.furnishing.replace('Unfurnishe', 'Unfurnished', inplace = True)
print('after replacement', df.furnishing.unique())

In [ ]:
print(ordinal_variables, end = ' ')
('Number of ordinal features is', len(ordinal_variables))

In [ ]:
fig, ax = plt.subplots(4,3, figsize = (20,30), sharey = False)
fig.tight_layout(pad= 4.0)
fig.subplots_adjust(top=0.95)

sns.set_style('white')
fig.suptitle('Ordinal Variable  vs Target Median', size = 30)
sns.barplot(ax= ax[0,0], data =df , x = df.groupby('bedroom').median().index, y =df.groupby('bedroom').median().rent,palette= 'muted' )
ax[0,0].set_title('Number of Bedrooms', fontsize = 15)

sns.barplot(ax= ax[0,1], data =df , x = df.groupby('bathrooms').median().index, y =df.groupby('bathrooms').median().rent,palette= 'muted' )
ax[0,1].set_title('Number of bathrooms', fontsize = 15)

sns.barplot(ax= ax[0,2], data =df , x = df.groupby('furnishing').median().index, y =df.groupby('furnishing').median().rent,palette= 'muted' )
ax[0,2].set_title('Furnishsing', fontsize = 15)

sns.barplot(ax= ax[1,0], data =df , x = df.groupby('avalable_for').median().index, y =df.groupby('avalable_for').median().rent,palette= 'muted' )
ax[1,0].set_title('Available for', fontsize = 15)

sns.barplot(ax= ax[1,1], data =df , x = df.groupby('floor_number').median().index, y =df.groupby('floor_number').median().rent,palette= 'muted' )
ax[1,1].set_title('Floor Number', fontsize = 15)

sns.barplot(ax= ax[1,2], data =df , x = df.groupby('facing').median().index, y =df.groupby('facing').median().rent,palette= 'muted' )
ax[1,2].set_title('', fontsize = 15)

sns.barplot(ax= ax[2,0], data =df , x = df.groupby('floor_type').median().index, y =df.groupby('floor_type').median().rent,palette= 'muted' )
ax[2,0].set_title('Floor Type', fontsize = 15)

sns.barplot(ax= ax[2,1], data =df , x = df.groupby('parking').median().index, y =df.groupby('parking').median().rent,palette= 'muted' )
ax[2,1].set_title('Count of Parking', fontsize = 15)

sns.barplot(ax= ax[2,2], data =df , x = df.groupby('aggDur').median().index, y =df.groupby('aggDur').median().rent,palette= 'muted' )
ax[2,2].set_title('Agreement Duration', fontsize = 15)

sns.barplot(ax= ax[3,0], data =df , x = df.groupby('noticeDur').median().index, y =df.groupby('noticeDur').median().rent,palette= 'muted' )
ax[3,0].set_title('Notice Duration', fontsize = 15)

sns.barplot(ax= ax[3,1], data =df , x = df.groupby('powerbackup').median().index, y =df.groupby('powerbackup').median().rent,palette= 'muted' )
ax[3,1].set_title('Power Backup Facility', fontsize = 15)

sns.barplot(ax= ax[3,2], data =df , x = df.groupby('propertyage').median().index, y =df.groupby('propertyage').median().rent,palette= 'muted' )
ax[3,2].set_title('Property Age', fontsize = 15)

fig.savefig('ordinal_variable.jpg', dpi = 100)

**From above visualization we  have concluded following points for higher rent.**
1. As number of bedroom increased rent also has incresed
2. As per furnishing concern Furinished has higher pay than unfurnished
3. Talking about direction if your home has east face your rent is higher as compare with south.
4. As floor number goes higher Rent Also increased
5. Wood Floor design(colors ) has higher pay than marble
6. As per powerbackup facility, if you have full backup == 2  then you need to pay more rent.

### Categorical Variables


In [ ]:
# ['address']
cat_variable = ['address']
df[cat_variable].head(2)

**Cleaning messy area & geting proper locality name from this**

In [ ]:
locality = []
for i in df.address:
    adds = i.split(',')
    if len(adds) == 4:
        locality.append(adds[0])
    elif len(adds) == 5:
        locality.append(adds[1])
    elif len(adds)  == 6:
        if len(adds[1]) == 1:
            locality.append(adds[2])
        else:
            locality.append(adds[1])
    elif len(adds) == 7:
        locality.append(' '.join(adds[2:4]))
    elif len(adds) == 8:
        locality.append(adds[2])
    else:
        locality.append(adds[4])

In [ ]:
df['locality'] = locality
location_summary = df['locality'].value_counts()
locations = [i.replace(',', ' ') for i in df.address]
len(location_summary.index)
location_summary.to_csv('summary.csv')

**This Summary.csv is a file of locality & based on that we are going to fetch latitude & logitude for those columns.**


I used goecoder library to get latitute & longotide for all locations as follows. & stored into a file **all_locations.csv** Which I will import here



**If you wanted to do all geoding please uncomment below code**

In [ ]:
# locations = list(df['locality'].value_counts().index)
# def get_latlng(neighborhood):
#     # initialize your variable to None
#     lat_lng_coords = None
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.arcgis('{} pune maaharashtra india'.format(neighborhood))
#         lat_lng_coords = g.latlng
#     return lat_lng_coords
# coords = list()
# for neighborhood in df2 ["Neighborhood"] .tolist():
#     coords.append(get_latlng(neighborhood))

In [ ]:
# map_data = pd.read_csv('all_location.csv')
# map_data.drop('Unnamed: 0', axis = 1, inplace =  True)
# map_data.head()
# lat_long_pune = [18.516726,73.856255]
# import folium

# def generateBaseMap(default_location= [18.516726,73.856255], default_zoom_start=11.5):
#     base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
#     return base_map
# base_map = generateBaseMap()
# ## add data to base map

# from folium.plugins import HeatMap
# base_map = generateBaseMap()

# gradient = {.33: 'black', .5: 'blue', 0.7: 'red'}
# HeatMap(data=map_data[['Longitude', 'Latitude', 'score']].groupby(['Longitude', 'Latitude']).sum().reset_index().values.tolist(), radius=8, gradient= gradient, max_zoom=13).add_to(base_map)
# # map_data.columns
# base_map.save('testmap.html')

**output of base map**
![map](map.jpg)

### Continous Variables

In [ ]:
# maintenace amt is similar to the mnt_amt & address has more unique values but it comes under categorical columnswe removed
#  both features & now we have area, brok_amt, deposit_amt & mnt_amount. 

continous_variable = continous_variable[0:1] +continous_variable[3:]


First we will look for uniques prices then we will check for missing & random  values 


In [ ]:
for i in continous_variable:
    print(i, len(df[i].unique()))

In [ ]:
 plt.plot(df.area) # looking for abnormality 

In [ ]:
print('shape before', df.shape)
df.drop(df[df['area'] > 15000].index, axis = 0, inplace = True)
print('shape after', df.shape)

In [ ]:
fig, ax =  plt.subplots(2,2, figsize=(20,15) )
sub = [(i,j)  for i in range(0,2) for j in range(0,2)]
for i, j in enumerate(continous_variable):
    sns.scatterplot(ax = ax[ sub[i][0],  sub[i][1]],  data = df, x = j, y = 'rent', hue= 'bedroom', size = 'bedroom', sizes = (20,300))

We have mostly left skewed data in all continous variables 

### Target Varible

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (20,8), sharey = False)
fig.tight_layout(pad= 4.0)
fig.subplots_adjust(top=0.87)
sns.set_style('white')
fig.suptitle('Target Varible', size = 30)
sns.histplot(ax= ax[0], data =df ,x = df.rent, kde =  True, bins = 15)
ax[0].set_title('Kdeplot', fontsize = 15)
sns.boxplot(ax= ax[1], data =df ,y  = df.rent, palette='muted')
ax[1].set_title('Boxplot', fontsize = 15)

fig.savefig('target.jpg', dpi = 250)

## Feature Engineering 

1. Outliers treatment
2. Feature transformation
3. Feature Scaling

### Outlier Treatment

In [ ]:
continous_variable

In [ ]:
just_test = df.describe().columns

In [ ]:
only_file = [i for i in just_test  if i not in binary_variables]
(only_file)

In [ ]:
fig, ax = plt.subplots(11,2, figsize = (20,30))
fig.tight_layout(pad = 4.0)
fig.subplots_adjust(top = 0.95)
fig.suptitle('Variable  Histplo & Box Plot', size = 30)
sns.kdeplot(ax= ax[0,0], data = df, x = 'bedroom')
sns.boxplot(ax= ax[0,1], data = df, x = 'bedroom')
sns.kdeplot(ax= ax[1,0], data = df, x = 'bathrooms')
sns.boxplot(ax= ax[1,1], data = df, x = 'bathrooms')
sns.kdeplot(ax= ax[2,0], data = df, x = 'area')
sns.boxplot(ax= ax[2,1], data = df, x = 'area')
sns.kdeplot(ax= ax[3,0], data = df, x = 'parking')
sns.boxplot(ax= ax[3,1], data = df, x = 'parking')
sns.kdeplot(ax= ax[4,0], data = df, x = 'aggDur')
sns.boxplot(ax= ax[4,1], data = df, x = 'aggDur')
sns.kdeplot(ax= ax[5,0], data = df, x = 'noticeDur')
sns.boxplot(ax= ax[5,1], data = df, x = 'noticeDur')
sns.kdeplot(ax= ax[6,0], data = df, x = 'powerbackup')
sns.boxplot(ax= ax[6,1], data = df, x = 'powerbackup')
sns.kdeplot(ax= ax[7,0], data = df, x = 'brok_amt')
sns.boxplot(ax= ax[7,1], data = df, x = 'brok_amt')
sns.kdeplot(ax= ax[8,0], data = df, x = 'deposit_amt')
sns.boxplot(ax= ax[8,1], data = df, x = 'deposit_amt')
sns.kdeplot(ax= ax[9,0], data = df, x = 'mnt_amt')
sns.boxplot(ax= ax[9,1], data = df, x = 'mnt_amt')
sns.kdeplot(ax= ax[10,0], data = df, x = 'powerbackup')
sns.boxplot(ax= ax[10,1], data = df, x = 'powerbackup')


Based on data following colums has most outliers & mostly left skewed data:
1. area
2. parking
3. brok_amt
4. seposit_amt
5. mnt_amt

& Little bit features engineering is needed on 
1. Bedroom
2. Bathrooms
3. aggDur
4. NoticeDur




In [ ]:
plt.figure(figsize= (20,10))

sns.heatmap(df.corr(), annot = True, cmap = 'PuBu')

adding one rs amount in all

In [ ]:
df['brok_amt'] = df['brok_amt'] + 1
df['deposit_amt'] = df['deposit_amt'] + 1
df['mnt_amt'] = df['mnt_amt'] + 1
df['rent'] = df['rent'] + 1

**Area**

In [ ]:
len(df[df['area'] == 0])
# we dont  have any mising data on area

Area has all positive data after apply log, sqrt  tranformation I found that we have positive data we can use boxcox transformation here

In [ ]:
from scipy import stats
fitted_area, _ = boxcox(df.area)
sns.histplot(fitted_area)
df['area'] =  fitted_area

## we are not ableo to remove all outliers but we are manage to merge 75  outliers out of 478  data.  Still 403  outliers are present in data
## Update data to area



In [ ]:
print('before',df.shape)
df.drop(['mnt_amt', 'parking'], axis =  True, inplace =  True)
print('after',df.shape)


In [ ]:
# APPLY log transformation on brok_amt 
df['brok_amt'] = np.log(df['brok_amt'])
sns.boxplot(df.brok_amt)

In [ ]:
# APPLY log transformation on deposit 
df['deposit_amt'] = np.log(df['deposit_amt'])
sns.boxplot(df.deposit_amt)

In [ ]:
df.head(2)

In [ ]:
print('before',df.shape)
df.drop(['maintenance_amt', 'address'], axis =  True, inplace =  True)
print('after',df.shape)


In [ ]:
df.head(2)

In [ ]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
lb, le, oe, ms,ss = LabelBinarizer(), LabelEncoder(), OrdinalEncoder, MinMaxScaler, StandardScaler

In [ ]:
bin_trans = [i for i in binary_variables  if i not in df.describe().columns] # geting binary variables & dtype is object
for column in bin_trans:
    df[column] = lb.fit_transform(df[column])

In [ ]:
## before going to further first work on locality
print('locality has {} unique values & it is huge'.format(len(df['locality'].unique())))

In [ ]:
df.locality.replace('', 'Pune', inplace =  True)


In [ ]:
df['address'] = df.locality
df['sqrt_depo'] = np.sqrt(df['bedroom'])
df.head(2)


In [ ]:
area_mean = {}

for area in np.unique(df.address):
#     print(area)
    area_mean[area] = df.loc[df['address'] == area]['sqrt_depo'].mean()
    
df['address_num'] = df['address'].map(area_mean)
df.head(2)

In [ ]:
print('shape of data  before', df.shape)
df.drop(columns=['address', 'sqrt_depo','locality'], inplace =  True)
print('shape of data  after', df.shape)

**Categorical Columns using labelencoder**

In [ ]:
for column in df.describe(include='object').columns:
    df[column] = le.fit_transform(df[column])

In [ ]:
df.describe().T

In [ ]:
## After all that we have completed feature engineering task

## Feature_selection 

In [ ]:
plt.figure(figsize = (20,15))
sns.heatmap(df.corr(), annot = True, cmap= 'BuPu')

In [ ]:
df.drop(columns = ['bathrooms'], inplace = True)

In [ ]:
### Next DAy model Builing 